DM-VTON

In [ ]:
!git clone https://github.com/ArefMYTB/DM-VTON.git
%cd DM-VTON

Cloning into 'DM-VTON'...
remote: Enumerating objects: 1496, done.
remote: Counting objects: 100% (563/563), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 1496 (delta 188), reused 345 (delta 163), pack-reused 933
Receiving objects: 100% (1496/1496), 3.94 MiB | 5.88 MiB/s, done.
Resolving deltas: 100% (585/585), done.
/content/DM-VTON


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 42.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [ ]:
# Download VITON-Clean dataset
!mkdir ../dataset
!gdown 1-5FtBJtel-ujgKR_TqJEcN2KrhyjBcyp -O ../dataset/VITON-Clean.zip
!unzip ../dataset/VITON-Clean.zip -d ../dataset
!rm ../dataset/VITON-Clean.zip

In [ ]:
!unzip ../dataset/test.zip -d ../dataset
!rm ../dataset/test.zip

In [ ]:
# Download checkpoints
!mkdir checkpoints
!gdown 16H16AhGJrqndDrAaHWZvTFzGSsaDecpS -O checkpoints/dmvton_pf_warp.pt
!gdown 1i-Ple5L9__LV_Fbx1cZyxiOC-CRFxi0C -O checkpoints/dmvton_pf_gen.pt

Downloading...
From (original): https://drive.google.com/uc?id=16H16AhGJrqndDrAaHWZvTFzGSsaDecpS
From (redirected): https://drive.google.com/uc?id=16H16AhGJrqndDrAaHWZvTFzGSsaDecpS&confirm=t&uuid=11492508-8097-4b24-87b5-66edd232b2ae
To: /content/DM-VTON/checkpoints/dmvton_pf_warp.pt
100% 86.5M/86.5M [00:02<00:00, 30.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1i-Ple5L9__LV_Fbx1cZyxiOC-CRFxi0C
From (redirected): https://drive.google.com/uc?id=1i-Ple5L9__LV_Fbx1cZyxiOC-CRFxi0C&confirm=t&uuid=2d9df1db-0a94-4dab-bc7a-1722d1baa3d8
To: /content/DM-VTON/checkpoints/dmvton_pf_gen.pt
100% 26.4M/26.4M [00:01<00:00, 22.6MB/s]


In [ ]:
import shutil

shutil.rmtree("/content/DM-VTON/runs")

In [ ]:
# Set batch_size = 1 to save each image separately
!python test.py --project runs/test --name DM-VTON_demo \
--device 0 --align_corners --batch_size 1 --workers 16 \
--dataroot ../dataset/test \
--pf_warp_checkpoint checkpoints/dmvton_pf_warp.pt \
--pf_gen_checkpoint checkpoints/dmvton_pf_gen.pt

In [ ]:
import shutil
from google.colab import files

# Path to the folder you want to download
folder_path = '/content/DM-VTON/images'

# Zip the folder
shutil.make_archive('/content/sample_folder', 'zip', folder_path)

# Download the zip file
files.download('/content/sample_folder.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bug in Colab

In [ ]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

Run here

# Imports

In [ ]:
import cupy
import torch

import torchvision.transforms as transforms
from PIL import Image, ImageDraw

from pipelines import DMVTONPipeline
from utils.general import Profile, print_log, warm_up
from utils.torch_utils import select_device

# RUN VTON

In [ ]:
# Device
device = select_device(0, batch_size=1)

# Inference Pipeline
pipeline = DMVTONPipeline(
    checkpoints={
        'warp': "checkpoints/dmvton_pf_warp.pt",
        'gen': "checkpoints/dmvton_pf_gen.pt",
    },
).to(device)
pipeline.eval()

In [ ]:
def run_test_pf(
    pipeline, real_image, clothes, edge, device
):

  # Warm-up gpu
    dummy_input = {
        'person': torch.randn(1, 3, 256, 192).to(device),
        'clothes': torch.randn(1, 3, 256, 192).to(device),
        'clothes_edge': torch.randn(1, 1, 256, 192).to(device),
    }
    with cupy.cuda.Device(int(device.split(':')[-1])):
        warm_up(pipeline, **dummy_input)

    with torch.no_grad():
        seen, dt = 0, Profile(device=device)
        # Prepare data
        real_image = real_image.to(device)
        clothes = clothes.to(device)
        edge = edge.to(device)

        with cupy.cuda.Device(int(device.split(':')[-1])):
            with dt:
                p_tryon, warped_cloth = pipeline(real_image, clothes, edge, phase="test")

        return p_tryon, warped_cloth


# Data loader functions

In [ ]:
def get_one(person):
  phase = "test"

  transform_image = get_transform(train=(phase == 'train'))
  transform_parse = get_transform(
      train=(phase == 'train'), method=Image.NEAREST, normalize=False
  )

  if type(person) == str:
    img = Image.open(person).convert('RGB')
  else:
    img = Image.fromarray(person).convert('RGB')
  img_tensor = transform_image(img)

  return img_tensor


def __getitem__(person, target, edge):


  phase = "test"

  transform_image = get_transform(train=(phase == 'train'))
  transform_parse = get_transform(
      train=(phase == 'train'), method=Image.NEAREST, normalize=False
  )

  if type(person) == str:
    img = Image.open(person).convert('RGB')
  else:
    imge = person.convert('RGB')
  img_tensor = transform_image(img)

  if type(target) == str:
    cloth = Image.open(target).convert('RGB')
  else:
    cloth = target.convert('RGB')
  cloth_tensor = transform_image(cloth)

  if type(edge) == str:
    cloth_edge = Image.open(edge).convert('L')
  else:
    cloth_edge = edge.convert('L')
  cloth_edge_tensor = transform_parse(cloth_edge)


  return img_tensor, cloth_tensor, cloth_edge_tensor



def get_transform(train, method=Image.BICUBIC, normalize=True):
    transform_list = []

    base = float(2**4)
    transform_list.append(transforms.Lambda(lambda img: __make_power_2(img, base, method)))

    if train:
        transform_list.append(transforms.Lambda(lambda img: __flip(img, 0)))

    transform_list += [transforms.ToTensor()]

    if normalize:
        transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)


def normalize():
    return transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))


def __make_power_2(img, base, method=Image.BICUBIC):
    try:
        ow, oh = img.size  # PIL
    except Exception:
        oh, ow = img.shape  # numpy
    h = int(round(oh / base) * base)
    w = int(round(ow / base) * base)
    if (h == oh) and (w == ow):
        return img
    return img.resize((w, h), method)


def __flip(img, flip):
    if flip:
        return img.transpose(Image.FLIP_LEFT_RIGHT)
    return img

# Test on image

In [ ]:
image_path = "/content/dataset/test/test_img/000057_0.jpg"
target_path = "/content/dataset/test/test_color/000523_1.jpg"
edge_path = "/content/dataset/test/test_edge/000523_1.jpg"

image_tensor, target_tensor, edge_tensor = __getitem__(image_path, target_path, edge_path)

image_tensor = image_tensor.unsqueeze(0)
target_tensor = target_tensor.unsqueeze(0)
edge_tensor = edge_tensor.unsqueeze(0)

p_tryon, warped_cloth = run_test_pf(
        pipeline=pipeline,
        real_image=image_tensor,
        clothes=target_tensor,
        edge=edge_tensor,
        device=device,
    )

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3809: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:464: UserWarning: cupy.cuda.compile_with_cache has been deprecated in CuPy v10, and will be removed in the future. Use cupy.RawModule or cupy.RawKernel instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
import torchvision as tv

In [ ]:
tv.utils.save_image(
                    p_tryon[0],
                    "test.jpg",
                    nrow=int(1),
                    normalize=True,
                    value_range=(-1, 1),
                )

# Test on Video

In [ ]:
import cv2
# from google.colab import files

# # Upload the video file
# uploaded = files.upload()

# # Load the video
# video_path = next(iter(uploaded))
video_path = "girl.mp4"
cap = cv2.VideoCapture(video_path)

output_size = (192, 256)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('flipped_video.avi', fourcc, fps, (width, height))

images = []
# Read and flip each frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Flip the frame horizontally
    # flipped_frame = cv2.flip(frame, 1)

    resized_frame = cv2.resize(frame, output_size)

    # try on
    img_tensor = get_one(resized_frame)
    # print(resized_frame.shape)
    # print(img_tensor.shape)
    img_tensor = img_tensor.unsqueeze(0)
    # print(img_tensor.shape)

    p_tryon, warped_cloth = run_test_pf(
        pipeline=pipeline,
        real_image=img_tensor,
        clothes=target_tensor,
        edge=edge_tensor,
        device=device,
    )

    images.append(p_tryon)


    # Write the flipped frame to the output video
    # out.write(resized_frame)

# Release everything if job is finished
cap.release()
# out.release()

print("Video flipping completed!")


In [ ]:
len(images)

314

In [ ]:
for j in range(len(images)):

                tv.utils.save_image(
                    images[j],
                    f"images/t{j}.jpg",
                    nrow=int(1),
                    normalize=True,
                    value_range=(-1, 1),
                )